In [1]:
import edward as ed
import tensorflow as tf
from keras.layers import Dense

Using TensorFlow backend.


In [56]:
??keras

In [2]:
from edward.models import Normal
from edward.models import Bernoulli
from scipy.misc import imsave

In [3]:
from tensorflow.contrib import slim
from tensorflow.examples.tutorials.mnist import input_data

In [28]:
M=100
d=2 # latent variable dimension

In [29]:
# prob model of VAE - the generative part. The latent variable posterior will be approximated by the variational model below
z=Normal(loc=tf.zeros([M,d]),scale=tf.ones([M,d]))
h=Dense(512,activation='relu')(z)
x=Bernoulli(logits=Dense(28*28,activation=None)(h),dtype=tf.float32)

In [30]:
# variational approx to the posterior over latent variables.
x_ph=tf.placeholder(tf.float32,[M,28*28])
qh=Dense(512,activation='relu')(x_ph) # assume that the parameters of the layer have a standard api and are updated via the inference algorithm
qz=Normal(loc=Dense(d,activation=None)(qh),scale=Dense(d,activation='softplus')(qh)) # fully connected layer to mean and extra non-linearity to variance

In [8]:
mnist=input_data.read_data_sets("data/mnist",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [31]:
data = {x: x_ph}


In [32]:
inference = ed.KLqp({z: qz},data)

In [33]:
optimizer=tf.train.RMSPropOptimizer(0.01,epsilon=1.0)
inference.initialize(optimizer=optimizer)

In [34]:
tf.initialize_all_variables().run()

In [ ]:
n_epoch=100
n_iter_per_epoch=5000
for _ in range(n_epoch):
    for _ in range(n_iter_per_epoch):
        x_train, _ = mnist.train.next_batch(M)
        info_dict=inference.update(feed_dict={x_ph:x_train})
    imgs=x.value().eval()
    for m in range(M):
        imsave("/home/mattma/Documents/data/img/%d.png" % m,imgs[m].reshape(28,28))
